In [1]:
import torch
import torch.nn as nn
import pandas as pd
import pyarrow as pa
import ast
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df = pd.read_csv('df_padding.csv')

In [3]:
df = df.drop("Unnamed: 0", axis=1)

In [4]:
df['mjd'] = df['mjd'].apply(ast.literal_eval)
df['magpsf_corr'] = df['magpsf_corr'].apply(ast.literal_eval)

In [5]:
mjd_tensor = torch.tensor(df['mjd'].tolist())
magpsf_corr_tensor = torch.tensor(df['magpsf_corr'].tolist())

In [6]:
tensor = torch.stack((mjd_tensor, magpsf_corr_tensor), dim=2)

torch.Size([287146, 150])

In [ ]:
N = 287146
zero_column = torch.zeros(N, 150, 1)

In [ ]:
mjd_tensor_with_zeros = torch.cat((mjd_tensor.unsqueeze(2), zero_column), dim=2)
magpsf_corr_tensor_with_zeros = torch.cat((zero_column, magpsf_corr_tensor.unsqueeze(2)), dim=2)

In [ ]:
tensor = mjd_tensor_with_zeros + magpsf_corr_tensor_with_zeros

In [ ]:
input_tensor = tensor.permute(0, 2, 1)

In [ ]:
input_tensor.shape

In [ ]:
input_data = input_tensor.unsqueeze(1)

In [ ]:
input_data.shape

In [ ]:
class Reshape(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


class Trim(nn.Module):
    def __init__(self, *args):
        super().__init__()

    def forward(self, x):
        return x[:, :, :2, :150]
    
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.01),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.01),
            nn.Flatten(),
            nn.Linear(3136, 64) 
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(2, 3136),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


autoencoder = AutoEncoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    outputs = autoencoder(input_data)
    loss = criterion(outputs, input_data)
    optimizer.zero_grad() 
    loss.backward()  
    optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')